In [1]:
import pandas as pd
import numpy as np
import os

filepath_common = 'proturn_output_'
strains = ['aj','balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso']
true_data_files = 'hl.out'
mask = [1, 2, 3, 4, 5]
folders = [filepath_common + s + '_' + e + '/' for s in strains for e in exp]
output_dir = 'nrmse_k'
os.makedirs(output_dir, exist_ok=True)


    

/Users/pinglab/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/pinglab/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:

for m in mask:
    sse_mask = 0
    nf_mask = 0
    for f in folders:
        print(f)
        true_path = f"{f}{true_data_files}"
        imputed_path = f"{f}ProTurn_imp_{m}.txt"
        
        true = pd.read_csv(true_path, sep='\t', low_memory=False)
        true = true.loc[(true['R2'] >= 0.8)]
        imputed_dmi = pd.read_csv(imputed_path, sep='\t', low_memory=False)
        imputed_dmi = imputed_dmi.loc[(imputed_dmi['R2'] >= 0.8)] #| (imputed_dmi['SS'] <= 0.05)]
        imputed_dmi = imputed_dmi.loc[(imputed_dmi['k'] >= 0) & (imputed_dmi['k'] <= 50)]
        
        sse = 0
        nf = 0
        
        for prot in imputed_dmi.Uniprot.unique():
            if prot not in true.Uniprot.unique():
                pass 
            else: 
                #imputed data for a specific protein
                imp_prot = imputed_dmi.loc[imputed_dmi['Uniprot'] == prot].groupby(['ID', 'Uniprot']).agg({'k': 'mean'}).reset_index()
                #true data for a specific protein
                t = true.loc[true['Uniprot'] == prot][['ID', 'Uniprot','k']]
                #find interesection of IDs between imputed and true data
                imp_prot = imp_prot.loc[imp_prot['ID'].isin(t['ID'])]
                t = t.loc[t['ID'].isin(imp_prot['ID'])]
                #print(imp_prot)
                #print(t)
            
                error = np.median(t['k']) - np.median(imp_prot['k'])
                sse += error**2
                nf += np.median(t['k'])**2
                #print(sse, nf)
        sse_mask += sse
        nf_mask += nf
    nrmse = np.sqrt(sse_mask/nf_mask) 
    print(f"{nrmse} for {m}")
    #with open(f'{output_dir}/nrmse_k.csv', 'a') as f:
    #            f.write(f'{nrmse},{m},dmi\n')
    break